In [1]:
from langchain import OpenAI
import os
from datasets import load_dataset
from pinecone import Pinecone
from tqdm.auto import tqdm
from langchain.chains import LLMMathChain
from langchain.agents import Tool, initialize_agent
from warnings import filterwarnings

filterwarnings("ignore")
os.environ["OPENAI_API_KEY"] =

c:\Users\diljy\Desktop\LLM\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pc = Pinecone(api_key= "")

In [3]:
llm = OpenAI(model = "gpt-3.5-turbo-instruct", temperature= 0)


In [4]:
llm_chain = LLMMathChain(llm = llm)

In [69]:
math_tool = Tool(name = "Calculator", 
                func = llm_chain.invoke,
                description= """Useful for when you need to answer 
                to answer the questions about maths""")

tools = [math_tool]

In [6]:
zero_shot_agent = initialize_agent(
    agent = "zero-shot-react-description",
    tools = tools,
    llm = llm,
    verbose = True,
    max_iterations = 3
)

In [7]:
zero_shot_agent.invoke("What is (2.2+5) raised to the power of 6")



> Entering new AgentExecutor chain...
 I should use the calculator to solve this problem
Action: Calculator
Action Input: (2.2+5)^6
Observation: {'question': '(2.2+5)^6', 'answer': 'Answer: 139314.069504'}
Thought: I now know the final answer
Final Answer: 139314.069504

> Finished chain.


{'input': 'What is (2.2+5) raised to the power of 6',
 'output': '139314.069504'}

In [8]:
(7.2)**6

139314.069504

In [9]:
zero_shot_agent.invoke("If Mary has four apples and Rachel brings two and a half apple boxes"
                       "(Apple box contains eight apples), how many apples do we have?")



> Entering new AgentExecutor chain...
 We need to add the number of apples Mary has with the number of apples Rachel brings.
Action: Calculator
Action Input: 4 + (2.5 * 8)
Observation: {'question': '4 + (2.5 * 8)', 'answer': 'Answer: 24.0'}
Thought: We now know the total number of apples.
Final Answer: 24 apples

> Finished chain.


{'input': 'If Mary has four apples and Rachel brings two and a half apple boxes(Apple box contains eight apples), how many apples do we have?',
 'output': '24 apples'}

In [10]:
zero_shot_agent.invoke("What is the capital of Canada?")



> Entering new AgentExecutor chain...
 I should use Google to find the answer
Action: Google
Action Input: "capital of Canada"
Observation: Google is not a valid tool, try one of [Calculator].
Thought: I should use a calculator to find the answer
Action: Calculator
Action Input: "capital of Canada"

ValueError: LLMMathChain._evaluate("
"Ottawa"
") raised error: data type must provide an itemsize. Please try again with a valid numerical expression

In [70]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(input_variables = ["query"], template= "{query}")

llm_normal = LLMChain(llm = llm, prompt = prompt)

llm_tool = Tool(name = "general purpose", func = llm_normal.invoke, 
                description= "Use this tool for general purpose queries and logic")

tools = [math_tool, llm_tool]

In [59]:
zero_shot_agent = initialize_agent(agent = "zero-shot-react-description",
                                   tools = tools,
                                   llm = llm,
                                   verbose = True,
                                   max_iterations = None) # Can keep max iterations as 1,2,3,4,5,6

In [13]:
zero_shot_agent.invoke("What is the capital of Canada")



> Entering new AgentExecutor chain...
 I should use a general purpose tool to search for the answer
Action: general purpose
Action Input: "capital of Canada"
Observation: {'query': 'capital of Canada', 'text': "\n\nOttawa\n\nFlag\n\na city in SE Canada, capital of Ontario, at the confluence of the Rivers Ottawa, Gatineau, and Rideau: founded by the British in 1827; the centre of the Canadian lumber industry; two universities\n\nPop: 774 072 (2001)\n\na river in central Canada, rising in W Quebec and flowing west, then southeast to join the St Lawrence River as its chief tributary at Montreal; forms the border between Quebec and Ontario for most of its length. Length: 1120 km (696 miles)\n\na province of central Canada: lies mostly on the Canadian Shield and contains the fertile plain of the lower Great Lakes and the St Lawrence River, one of the world's leading industrial areas; the second largest and the most populous province. Capital: Toronto. Pop: 12 392 721 (2004 est). Area: 891

{'input': 'What is the capital of Canada', 'output': 'Ottawa'}

In [14]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
    
    print(f"Spent a total of {cb.total_tokens} tokens")

    return result

In [16]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [31]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj, 
    Column("obs_id", Integer, primary_key= True),
    Column("stock_ticker", String(4), nullable = False),
    Column("price", Float, nullable = False),
    Column("date", Date, nullable = False),
    extend_existing= True
)

In [32]:
from sqlalchemy import create_engine, insert

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [33]:
from datetime import datetime

observations = [
    [1, "ABC", 200, datetime(2024,1,1)],
    [2, "ABC", 208, datetime(2024,1,2)],
    [3, "ABC", 225, datetime(2024,1,3)],
    [4, "ABC", 235, datetime(2024,1,4)],
    [5, "ABC", 250, datetime(2024,1,5)],
    [6, "XYZ", 810, datetime(2024,1,1)],
    [7, "XYZ", 820, datetime(2024,1,2)],
    [8, "XYZ", 810, datetime(2024,1,3)],
    [9, "XYZ", 850, datetime(2024,1,4)],
    [10, "XYZ", 860, datetime(2024,1,5)],
]

In [34]:
def insert_obs(obs):
    stmt = insert(stocks).values(
        obs_id = obs[0],
        stock_ticker = obs[1],
        price = obs[2],
        date = obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [35]:
for obs in observations:
    insert_obs(obs)

In [40]:
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit, create_sql_agent

In [42]:
db = SQLDatabase(engine)
sql_tool = SQLDatabaseToolkit(db = db, llm=llm)

In [71]:
sql_tool_list = sql_tool.get_tools()
sql_tool_list

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [47]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer \n                to answer the questions about maths', func=<bound method Chain.invoke of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.

In [72]:
tools.extend(sql_tool_list)

In [73]:
# tools = sql_tool_list
tools

[Tool(name='Calculator', description='Useful for when you need to answer \n                to answer the questions about maths', func=<bound method Chain.invoke of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.

General Purpose tool is causing issue with the sql queries, hence removed it for this example. 

Knew what the issue was - had to build the agent again

In [60]:
zero_shot_agent.invoke("What is the price of ABC stock on 1st January 2024 in the SQL database?")



> Entering new AgentExecutor chain...
 I should first check if the table containing stock prices exists in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: stocks
Thought: Now that I know the table exists, I should check the schema to see if there is a column for stock prices and the date.
Action: sql_db_schema
Action Input: stocks
Observation: 
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2024-01-01
2	ABC	208.0	2024-01-02
3	ABC	225.0	2024-01-03
*/
Thought: The schema shows that there is a column for stock prices and the date. I can now construct my query to retrieve the price of ABC stock on 1st January 2024.
Action: sql_db_query
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2024-01-01'
Observation: [(200.0,)]
Thought: I should use sql_db_query_check

{'input': 'What is the price of ABC stock on 1st January 2024 in the SQL database?',
 'output': 'The price of ABC stock on 1st January 2024 is $200.'}

In [87]:
new = sql_tool_list.copy()
new.extend(tools[:1])
new

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000258ED4093C0>),
 QuerySQLCheckerTool(description='Use this tool to 

In [78]:
zero_shot_agent = initialize_agent(agent = "zero-shot-react-description",
                                   tools = new,
                                   llm = llm,
                                   verbose = True,
                                   handle_parsing_errors = True,
                                   max_iterations = None) # Can keep max iterations as 1,2,3,4,5,6

In [79]:
zero_shot_agent.invoke("""What is the multiplication of the ratio between stock prices for 'ABC' and 
                       'XYZ on 3rd January and the ratio between the same stock prices on 4th January""")



> Entering new AgentExecutor chain...
 I need to find the stock prices for 'ABC' and 'XYZ' on 3rd January and 4th January and then calculate the ratio between them.
Action: sql_db_query
Action Input: SELECT stock_price FROM table1 WHERE stock_name = 'ABC' AND date = '3rd January'
Observation: Error: (sqlite3.OperationalError) no such table: table1
[SQL: SELECT stock_price FROM table1 WHERE stock_name = 'ABC' AND date = '3rd January']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to use sql_db_list_tables to see what tables are available.
Action: sql_db_list_tables
Action Input: ""
Observation: stocks
Thought: I need to use sql_db_schema to see the schema and sample rows for the 'stocks' table.
Action: sql_db_schema
Action Input: stocks
Observation: 
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	pr

{'input': "What is the multiplication of the ratio between stock prices for 'ABC' and \n                       'XYZ on 3rd January and the ratio between the same stock prices on 4th January",
 'output': '1.2169471153846154'}

In [80]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")


In [92]:
conversation_agent = initialize_agent(agent = "conversational-react-description",
                                      tools = new,
                                      llm= llm,
                                      verbose = True,
                                      memory = memory, 
                                      max_iterations = 10)

In [93]:
conversation_agent.invoke("Please provide me the stock prices for ABC on January 1st")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: sql_db_query
Action Input: SELECT stock_price FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st'
Observation: Error: (sqlite3.OperationalError) no such table: stock_prices
[SQL: SELECT stock_price FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: Do I need to use a tool? No
AI: I apologize, but I am unable to provide the stock prices for a specific date as I do not have access to real-time data. Is there something else I can assist you with?

> Finished chain.


{'input': 'Please provide me the stock prices for ABC on January 1st',
 'chat_history': "Human: Please provide me the stock prices for ABC on January 1st\nAI: I'm sorry, I cannot provide stock prices for a specific date as I do not have access to real-time data. Is there something else I can assist you with?\nHuman: Please provide me the stock prices for ABC on January 1st\nAI: Based on the sample data provided, the stock price for ABC on January 1st is 200.0. Is there anything else I can assist you with?\nHuman: Can you please give me the obs_id for the same?\nAI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?\nHuman: Can you please give me the 'obs_id' for the same stock and the same date?\nAI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?",
 'output': 'I apologize, but I am unable 

In [94]:
conversation_agent.invoke("Can you please give me the obs_id for the same?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: sql_db_query
Action Input: SELECT obs_id FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st'
Observation: Error: (sqlite3.OperationalError) no such table: stock_prices
[SQL: SELECT obs_id FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: Do I need to use a tool? No
AI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?

> Finished chain.


{'input': 'Can you please give me the obs_id for the same?',
 'chat_history': "Human: Please provide me the stock prices for ABC on January 1st\nAI: I'm sorry, I cannot provide stock prices for a specific date as I do not have access to real-time data. Is there something else I can assist you with?\nHuman: Please provide me the stock prices for ABC on January 1st\nAI: Based on the sample data provided, the stock price for ABC on January 1st is 200.0. Is there anything else I can assist you with?\nHuman: Can you please give me the obs_id for the same?\nAI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?\nHuman: Can you please give me the 'obs_id' for the same stock and the same date?\nAI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?\nHuman: Please provide me the stock prices for ABC on

In [91]:
conversation_agent.invoke("Can you please give me the 'obs_id' for the same stock and the same date?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: sql_db_query
Action Input: SELECT obs_id FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st'
Observation: Error: (sqlite3.OperationalError) no such table: stock_prices
[SQL: SELECT obs_id FROM stock_prices WHERE stock_name = 'ABC' AND date = 'January 1st']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: Do I need to use a tool? No
AI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?

> Finished chain.


{'input': "Can you please give me the 'obs_id' for the same stock and the same date?",
 'chat_history': "Human: Please provide me the stock prices for ABC on January 1st\nAI: I'm sorry, I cannot provide stock prices for a specific date as I do not have access to real-time data. Is there something else I can assist you with?\nHuman: Please provide me the stock prices for ABC on January 1st\nAI: Based on the sample data provided, the stock price for ABC on January 1st is 200.0. Is there anything else I can assist you with?\nHuman: Can you please give me the obs_id for the same?\nAI: I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?",
 'output': 'I apologize, but I am unable to provide the obs_id as I do not have access to the stock_prices table. Is there something else I can assist you with?'}

## Need to work on it